[![Renode](https://dl.antmicro.com/projects/renode/renode.svg)](https://renode.io)

[![Run in Google Colab](https://img.shields.io/badge/-Run%20in%20Google%20colab-%23007ded?logo=google-colab&logoColor=white&style=for-the-badge)](https://colab.research.google.com/github/antmicro/test-colabs/blob/main/boards/nrf5340_audio_dk_nrf5340_cpuapp_blinky.ipynb) [![View ipynb](https://img.shields.io/badge/-View%20ipynb%20source-%23007ded?logo=jupyter&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/nrf5340_audio_dk_nrf5340_cpuapp_blinky.ipynb) [![View Python source](https://img.shields.io/badge/-View%20Python%20source-%23007ded?logo=python&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/nrf5340_audio_dk_nrf5340_cpuapp_blinky.py)

## Install requirements

In [ ]:
! pip install -q git+https://github.com/antmicro/renode-colab-tools.git
! pip install -q git+https://github.com/antmicro/renode-run.git
! pip install -q git+https://github.com/antmicro/pyrenode.git
! renode-run download

## Start Renode

In [ ]:
from pyrenode import connect_renode, get_keywords
connect_renode()
get_keywords()

## Setup a script

In [ ]:
%%writefile script.resc

using sysbus
$name?="nrf5340_audio_dk_nrf5340_cpuapp"
mach create $name

machine LoadPlatformDescription @https://zephyr-dashboard.renode.io/nrf5340_audio_dk_nrf5340_cpuapp-blinky/nrf5340_audio_dk_nrf5340_cpuapp-blinky.repl
machine EnableProfiler $ORIGIN/metrics.dump

showAnalyzer sysbus.uart0
sysbus.uart0 RecordToAsciinema $ORIGIN/output.asciinema

macro reset
"""
    sysbus LoadELF @https://new-zephyr-dashboard.renode.io/zephyr/3f655524463c8310943611fbeee34a66436b16ab/nrf5340_audio_dk_nrf5340_cpuapp/blinky/blinky.elf
    cpu0 VectorTableOffset `sysbus GetSymbolAddress "_vector_table"`
"""

runMacro $reset

## Run the sample

In [ ]:
ExecuteScript("script.resc")
CreateTerminalTester("sysbus.uart0", timeout=5)
ExecuteCommand('emulation CreateLEDTester "led_tester" "sysbus.gpio0.rgb1red"')
StartEmulation()

WaitForLineOnUart("Booting Zephyr OS", treatAsRegex=True)
ExecuteCommand("led_tester AssertState true 1")
ExecuteCommand("led_tester AssertState false 1")
ExecuteCommand("led_tester AssertState true 1")
ExecuteCommand("led_tester AssertState false 1")
ExecuteCommand("led_tester AssertState true 1")
ExecuteCommand("led_tester AssertState false 1")

ResetEmulation()

## UART output

In [ ]:
from renode_colab_tools import asciinema
asciinema.display_asciicast('output.asciinema')

## Renode metrics analysis

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path('/root/.config/renode/renode-run.path').read_text())

from renode_colab_tools import metrics
from tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('metrics.dump')

metrics.display_metrics(parser)